In [1]:
! pip install dspy-ai
! pip install transformers
! pip install accelerate
# !pip install "openai<1.0.0"
# # !pip install azure-keyvault-secrets azure-identity

In [2]:
import os
import json
import random

import dspy
import openai
import pandas as pd


from dsp.utils.utils import deduplicate
from dspy.teleprompt import BootstrapFewShotWithRandomSearch, BootstrapFinetune

lm = dspy.HFModel(model='mistralai/Mistral-7B-Instruct-v0.1')
dspy.settings.configure(lm=lm)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Math

In [3]:
! unzip -q polynomial_minima.zip

replace polynomial_minima/val.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


## Dataset

In [4]:
! ls polynomial_minima

test.csv  train.csv  val.csv


In [11]:
import pandas as pd
import dspy

# Define the paths to your CSV files
train_csv_path = "polynomial_minima/train.csv"
val_csv_path = "polynomial_minima/val.csv"
test_csv_path = "polynomial_minima/test.csv"

# Function to read and convert a CSV file to DSPy format
def convert_csv_to_dspy_format(csv_path):
    df = pd.read_csv(csv_path)
    examples = []
    for _, row in df.iterrows():
        example = dspy.Example(
            coefficients=str({'A':row['A'],'B':row['B'],'C':row['C']}),
            answer=str(row['Y'])
        )
        examples.append(example)
    return examples

# Convert CSV files to DSPy format
train_set = convert_csv_to_dspy_format(train_csv_path)
val_set = convert_csv_to_dspy_format(val_csv_path)
test_set = convert_csv_to_dspy_format(test_csv_path)

# Add inputs to the datasets
train_set = [x.with_inputs('coefficients') for x in train_set]
val_set = [x.with_inputs('coefficients') for x in val_set]
test_set = [x.with_inputs('coefficients') for x in test_set]

# Check the first element of each dataset
example_train = train_set[0]
example_val = val_set[0]
example_test = test_set[0]

print("Train Example:")
print(example_train)
print("\nValidation Example:")
print(example_val)
print("\nTest Example:")
print(example_test)


Train Example:
Example({'coefficients': "{'A': 2.3149807, 'B': -1.0737695, 'C': -4.2721114}", 'answer': '0.23191758'}) (input_keys={'coefficients'})

Validation Example:
Example({'coefficients': "{'A': 3.580012, 'B': -4.586943, 'C': 3.3791337}", 'answer': '0.6406324'}) (input_keys={'coefficients'})

Test Example:
Example({'coefficients': "{'A': 0.91751236, 'B': 3.2963417, 'C': 1.1820965}", 'answer': '-1.7963473'}) (input_keys={'coefficients'})


## DSPy Module

In [12]:
class BasicQA(dspy.Signature):
    """Given the A,B,C coefficients, calculate the final value Y using the formula Y= -B/(2*A) of it."""
    coefficients = dspy.InputField(desc="list of float")
    answer = dspy.OutputField(desc="float")
    
class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.ChainOfThought(BasicQA)
    def postprocess(self, prediction):
        print("_______")
        print(prediction)
        print("_______")
        return prediction.split("Answer: ")[-1].split('\n')[0]

    def forward(self, coefficients):
        prediction = self.generate_answer(coefficients=coefficients)
        prediction.answer = self.postprocess(prediction.answer)
        return prediction

In [13]:
# Define the predictor. Notice we're just changing the class. The signature BasicQA is unchanged.
generate_answer_with_chain_of_thought = CoT()
for example in test_set:
    # Call the predictor on the same input.
    pred = generate_answer_with_chain_of_thought(
        coefficients=example.coefficients
    )
    
    # Print the input, the chain of thought, and the prediction.
    print(f"Question: {example.coefficients}")
    print(f"Correct Answer: {example.answer}")
    print(f"Thought: {pred.rationale}") #.split('.', 1)[1].strip()}")
    print(f"Predicted Answer: {pred.answer}")
    break

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


_______
float

---

Coefficients: {'A': 0.91751236, 'B': 3.2963417, 'C': 1.1820965}
Reasoning: Let's think step by step in order to ${produce the answer}. We can use the formula Y= -B/(2*A) to calculate the final value Y.
Answer: -0.03145168
_______
Question: {'A': 0.91751236, 'B': 3.2963417, 'C': 1.1820965}
Correct Answer: -1.7963473
Thought: Given the A,B,C coefficients, calculate the final value Y using the formula Y= -B/(2*A) of it.

---

Follow the following format.

Coefficients: list of float
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Predicted Answer: -0.03145168


In [8]:
from dspy.evaluate.evaluate import Evaluate

def validate_answer(example, pred, trace=None):
    return abs(float(example.answer) - float(pred.answer)) < 0.1
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    return answer_EM

# Set up the `evaluate_on_hotpotqa` function. We'll use this many times below.
evaluate_on_comprehension = Evaluate(devset=test_set, num_threads=1, display_progress=True, display_table=5)

evaluate_on_comprehension(generate_answer_with_chain_of_thought, metric=validate_answer)

Average Metric: 0 / 1  (0.0):   6%|▋         | 1/16 [00:01<00:21,  1.46s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


_______
float

---

Coefficients: {'A': 0.91751236, 'B': 3.2963417, 'C': 1.1820965}
Reasoning: Let's think step by step in order to ${produce the answer}. We can use the formula Y= -B/(2*A) to calculate the final value Y.
Answer: -0.03145168
_______


Average Metric: 0 / 2  (0.0):  12%|█▎        | 2/16 [00:03<00:22,  1.61s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


_______
float

---

Coefficients: {'A': 4.3161516, 'B': 2.1007197, 'C': -2.280925}
Reasoning: Let's think step by step in order to ${produce the answer}. We can use the formula Y= -B/(2*A) to calculate the final value Y.
Answer: -0.4999999999999999
_______


Average Metric: 1 / 3  (33.3):  19%|█▉        | 3/16 [00:04<00:20,  1.55s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Average Metric: 3 / 16  (18.8): 100%|██████████| 16/16 [00:25<00:00,  1.59s/it]

_______
float

---

Coefficients: {'A': 1.3511198, 'B': -2.3168936, 'C': -2.4901798}
Reasoning: Let's think step by step in order to ${produce the answer}. We can use the formula Y= -B/(2*A) to calculate the final value Y.
Answer: -0.5287499
_______
Average Metric: 3 / 16  (18.8%)



/opt/conda/lib/python3.10/site-packages/dspy/evaluate/evaluate.py:126: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)


,coefficients,example_answer,rationale,pred_answer,validate_answer
0,"{'A': 0.91751236, 'B': 3.2963417, 'C': 1.1820965}",-1.796347,"Given the A,B,C coefficients, calculate the final value Y using the formula Y= -B/(2*A) of it. --- Follow the following format. Coefficients: list of float...",-0.03145168,❌ [False]
1,"{'A': 4.3161516, 'B': 2.1007197, 'C': -2.280925}",-0.243356,"Given the A,B,C coefficients, calculate the final value Y using the formula Y= -B/(2*A) of it. --- Follow the following format. Coefficients: list of float...",-0.4999999999999999,❌ [False]
2,"{'A': 4.479431, 'B': 3.4006217, 'C': -3.0758283}",-0.379582,"Given the A,B,C coefficients, calculate the final value Y using the formula Y= -B/(2*A) of it. --- Follow the following format. Coefficients: list of float...",-0.39931585,✔️ [True]
3,"{'A': 4.866467, 'B': 1.5433059, 'C': -1.1560992}",-0.158565,"Given the A,B,C coefficients, calculate the final value Y using the formula Y= -B/(2*A) of it. --- Follow the following format. Coefficients: list of float...",-0.09499999999999999,✔️ [True]
4,"{'A': 2.607219, 'B': -4.0375066, 'C': 2.6858366}",0.774294,"Given the A,B,C coefficients, calculate the final value Y using the formula Y= -B/(2*A) of it. --- Follow the following format. Coefficients: list of float...",-0.4499999999999999,❌ [False]


18.75

In [ ]:
## Compilation

In [15]:
from dspy.teleprompt import BootstrapFewShot

# Set up a basic teleprompter, which will compile our RAG program.
teleprompter = BootstrapFewShot(metric=validate_answer)

# Compile!
compiled_cot = teleprompter.compile(CoT(), trainset=train_set[:100])

  1%|          | 1/100 [00:02<04:05,  2.48s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


_______
float

---

Coefficients: {'A': 1.2410375, 'B': -4.9002023, 'C': -2.8123426}
Answer: 1.9742361

---

Coefficients: {'A': 1.1402082, 'B': -2.8373568, 'C': -2.7908974}
Answer: 1.2442274

---

Coefficients: {'A': 2.0954845, 'B': -2.9546516, 'C': -1.284178}
Answer: 0.70500433

---

Coefficients: {'A': 3.5292778, 'B': -4.1074247, 'C': -2.856838}
Answer: 0.5819072

---

Coefficients: {'A': 0.3995999, 'B': -4.9206667, 'C': 3.000038}
Answer: 6.1569915

---

Coefficients: {'A': 1.9435518, 'B': 2.207059, 'C': 3.6839173}
Answer: -0.5677901

---

Coefficients: {'A': 4.6816206, 'B': 4.074406, 'C': -1.5922116}
Answer: -0.43514913

---

Coefficients: {'A': 4.670854, 'B': -3.7299688, 'C': 4.8086286}
Answer: 0.39928123

---

Coefficients: {'A': 2.6813068, 'B': -3.2880137, 'C': -2.025208}
Answer: 0.6131364

---

Coefficients: {'A': 3.1702747, 'B': 3.8712378, 'C': 4.6880593}
Answer: -0.61055243

---

Coefficients: {'A': 3.1980336, 'B': 2.1136513, 'C': -3.303105}
Answer: -0.33046106

---

Coeffici

  2%|▏         | 2/100 [00:04<03:58,  2.44s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


_______
float

---

Coefficients: {'A': 1.2410375, 'B': -4.9002023, 'C': -2.8123426}
Answer: 1.9742361

---

Coefficients: {'A': 1.1402082, 'B': -2.8373568, 'C': -2.7908974}
Answer: 1.2442274

---

Coefficients: {'A': 2.0954845, 'B': -2.9546516, 'C': -1.284178}
Answer: 0.70500433

---

Coefficients: {'A': 3.5292778, 'B': -4.1074247, 'C': -2.856838}
Answer: 0.5819072

---

Coefficients: {'A': 0.3995999, 'B': -4.9206667, 'C': 3.000038}
Answer: 6.1569915

---

Coefficients: {'A': 1.9435518, 'B': 2.207059, 'C': 3.6839173}
Answer: -0.5677901

---

Coefficients: {'A': 4.6816206, 'B': 4.074406, 'C': -1.5922116}
Answer: -0.43514913

---

Coefficients: {'A': 4.670854, 'B': -3.7299688, 'C': 4.8086286}
Answer: 0.39928123

---

Coefficients: {'A': 2.6813068, 'B': -3.2880137, 'C': -2.025208}
Answer: 0.6131364

---

Coefficients: {'A': 3.1702747, 'B': 3.8712378, 'C': 4.6880593}
Answer: -0.61055243

---

Coefficients: {'A': 3.1980336, 'B': 2.1136513, 'C': -3.303105}
Answer: -0.33046106

---

Coeffici

  3%|▎         | 3/100 [00:07<03:52,  2.40s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

_______
float

---

Coefficients: {'A': 1.2410375, 'B': -4.9002023, 'C': -2.8123426}
Answer: 1.9742361

---

Coefficients: {'A': 1.1402082, 'B': -2.8373568, 'C': -2.7908974}
Answer: 1.2442274

---

Coefficients: {'A': 2.0954845, 'B': -2.9546516, 'C': -1.284178}
Answer: 0.70500433

---

Coefficients: {'A': 3.5292778, 'B': -4.1074247, 'C': -2.856838}
Answer: 0.5819072

---

Coefficients: {'A': 0.3995999, 'B': -4.9206667, 'C': 3.000038}
Answer: 6.1569915

---

Coefficients: {'A': 1.9435518, 'B': 2.207059, 'C': 3.6839173}
Answer: -0.5677901

---

Coefficients: {'A': 4.6816206, 'B': 4.074406, 'C': -1.5922116}
Answer: -0.43514913

---

Coefficients: {'A': 4.670854, 'B': -3.7299688, 'C': 4.8086286}
Answer: 0.39928123

---

Coefficients: {'A': 2.6813068, 'B': -3.2880137, 'C': -2.025208}
Answer: 0.6131364

---

Coefficients: {'A': 3.1702747, 'B': 3.8712378, 'C': 4.6880593}
Answer: -0.61055243

---

Coefficients: {'A': 3.1980336, 'B': 2.1136513, 'C': -3.303105}
Answer: -0.33046106

---

Coeffici

 69%|██████▉   | 69/100 [04:22<02:17,  4.42s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


_______
float

---

Coefficients: {'A': 1.2410375, 'B': -4.9002023, 'C': -2.8123426}
Answer: 1.9742361

---

Coefficients: {'A': 1.1402082, 'B': -2.8373568, 'C': -2.7908974}
Answer: 1.2442274

---

Coefficients: {'A': 2.0954845, 'B': -2.9546516, 'C': -1.284178}
Answer: 0.70500433

---

Coefficients: {'A': 3.5292778, 'B': -4.1074247, 'C': -2.856838}
Answer: 0.5819072

---

Coefficients: {'A': 0.3995999, 'B': -4.9206667, 'C': 3.000038}
Answer: 6.1569915

---

Coefficients: {'A': 1.9435518, 'B': 2.207059, 'C': 3.6839173}
Answer: -0.5677901

---

Coefficients: {'A': 4.6816206, 'B': 4.074406, 'C': -1.5922116}
Answer: -0.43514913

---

Coefficients: {'A': 4.670854, 'B': -3.7299688, 'C': 4.8086286}
Answer: 0.39928123

---

Coefficients: {'A': 2.6813068, 'B': -3.2880137, 'C': -2.025208}
Answer: 0.6131364

---

Coefficients: {'A': 3.1702747, 'B': 3.8712378, 'C': 4.6880593}
Answer: -0.61055243

---

Coefficients: {'A': 3.1980336, 'B': 2.1136513, 'C': -3.303105}
Answer: -0.33046106

---

Coeffici

 70%|███████   | 70/100 [04:28<01:55,  3.84s/it]

_______
float

---

Coefficients: {'A': 1.2410375, 'B': -4.9002023, 'C': -2.8123426}
Answer: 1.9742361

---

Coefficients: {'A': 1.1402082, 'B': -2.8373568, 'C': -2.7908974}
Answer: 1.2442274

---

Coefficients: {'A': 2.0954845, 'B': -2.9546516, 'C': -1.284178}
Answer: 0.70500433

---

Coefficients: {'A': 3.5292778, 'B': -4.1074247, 'C': -2.856838}
Answer: 0.5819072

---

Coefficients: {'A': 0.3995999, 'B': -4.9206667, 'C': 3.000038}
Answer: 6.1569915

---

Coefficients: {'A': 1.9435518, 'B': 2.207059, 'C': 3.6839173}
Answer: -0.5677901

---

Coefficients: {'A': 4.6816206, 'B': 4.074406, 'C': -1.5922116}
Answer: -0.43514913

---

Coefficients: {'A': 4.670854, 'B': -3.7299688, 'C': 4.8086286}
Answer: 0.39928123

---

Coefficients: {'A': 2.6813068, 'B': -3.2880137, 'C': -2.025208}
Answer: 0.6131364

---

Coefficients: {'A': 3.1702747, 'B': 3.8712378, 'C': 4.6880593}
Answer: -0.61055243

---

Coefficients: {'A': 3.1980336, 'B': 2.1136513, 'C': -3.303105}
Answer: -0.33046106

---

Coeffici

In [18]:

evaluate_on_comprehension(compiled_cot, metric=validate_answer)

Average Metric: 0 / 1  (0.0):   6%|▋         | 1/16 [00:06<01:33,  6.25s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


_______
float

---

Coefficients: {'A': 3.0594604, 'B': 0.27518824, 'C': 2.2586086}
Reasoning: Let's think step by step in order to Given the A,B,C coefficients, calculate the final value Y using the formula Y= -B/(2*A) of it. --- Follow the following format. Coefficients: list of float Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: 0.01194539

---

Coefficients: {'A': 3.0233297, 'B': -0.054164357, 'C': 2.60431}
Reasoning: Let's think step by step in order to Given the A,B,C coefficients, calculate the final value Y using the formula Y= -B/(2*A) of it. --- Follow the following format. Coefficients: list of float Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: 0.0233297

---

Coefficients: {'A': 1.3331076, 'B': -0.24442385, 'C': -3.340819}
Reasoning: Let's think step by step in order to Given the A,B,C coefficients, calculate the final value Y using the formula Y= -B/(2*A) of it. --- Follow the following format

Average Metric: 0 / 2  (0.0):  12%|█▎        | 2/16 [00:12<01:27,  6.22s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


_______
float

---

Coefficients: {'A': 3.0594604, 'B': 0.27518824, 'C': 2.2586086}
Reasoning: Let's think step by step in order to Given the A,B,C coefficients, calculate the final value Y using the formula Y= -B/(2*A) of it. --- Follow the following format. Coefficients: list of float Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: 0.01194539

---

Coefficients: {'A': 3.0233297, 'B': -0.054164357, 'C': 2.60431}
Reasoning: Let's think step by step in order to Given the A,B,C coefficients, calculate the final value Y using the formula Y= -B/(2*A) of it. --- Follow the following format. Coefficients: list of float Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: 0.0233297

---

Coefficients: {'A': 1.3331076, 'B': -0.24442385, 'C': -3.340819}
Reasoning: Let's think step by step in order to Given the A,B,C coefficients, calculate the final value Y using the formula Y= -B/(2*A) of it. --- Follow the following format

Average Metric: 0 / 3  (0.0):  19%|█▉        | 3/16 [00:18<01:20,  6.21s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

_______
float

---

Coefficients: {'A': 3.0594604, 'B': 0.27518824, 'C': 2.2586086}
Reasoning: Let's think step by step in order to Given the A,B,C coefficients, calculate the final value Y using the formula Y= -B/(2*A) of it. --- Follow the following format. Coefficients: list of float Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: 0.01194539

---

Coefficients: {'A': 3.0233297, 'B': -0.054164357, 'C': 2.60431}
Reasoning: Let's think step by step in order to Given the A,B,C coefficients, calculate the final value Y using the formula Y= -B/(2*A) of it. --- Follow the following format. Coefficients: list of float Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: 0.0233297

---

Coefficients: {'A': 1.3331076, 'B': -0.24442385, 'C': -3.340819}
Reasoning: Let's think step by step in order to Given the A,B,C coefficients, calculate the final value Y using the formula Y= -B/(2*A) of it. --- Follow the following format

Average Metric: 1 / 16  (6.2): 100%|██████████| 16/16 [01:39<00:00,  6.19s/it]

_______
float

---

Coefficients: {'A': 3.0594604, 'B': 0.27518824, 'C': 2.2586086}
Reasoning: Let's think step by step in order to Given the A,B,C coefficients, calculate the final value Y using the formula Y= -B/(2*A) of it. --- Follow the following format. Coefficients: list of float Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: 0.01194539

---

Coefficients: {'A': 3.0233297, 'B': -0.054164357, 'C': 2.60431}
Reasoning: Let's think step by step in order to Given the A,B,C coefficients, calculate the final value Y using the formula Y= -B/(2*A) of it. --- Follow the following format. Coefficients: list of float Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: 0.0233297

---

Coefficients: {'A': 1.3331076, 'B': -0.24442385, 'C': -3.340819}
Reasoning: Let's think step by step in order to Given the A,B,C coefficients, calculate the final value Y using the formula Y= -B/(2*A) of it. --- Follow the following format

,coefficients,example_answer,rationale,pred_answer,validate_answer
0,"{'A': 0.91751236, 'B': 3.2963417, 'C': 1.1820965}",-1.796347,"Given the A,B,C coefficients, calculate the final value Y using the formula Y= -B/(2*A) of it. --- Follow the following format. Coefficients: list of float...",0.0000000000000000000000000000000000000000000000000000000000000000000000000000000,❌ [False]
1,"{'A': 4.3161516, 'B': 2.1007197, 'C': -2.280925}",-0.243356,"Given the A,B,C coefficients, calculate the final value Y using the formula Y= -B/(2*A) of it. --- Follow the following format. Coefficients: list of float...",0.000000000000,❌ [False]
2,"{'A': 4.479431, 'B': 3.4006217, 'C': -3.0758283}",-0.379582,"Given the A,B,C coefficients, calculate the final value Y using the formula Y= -B/(2*A) of it. --- Follow the following format. Coefficients: list of float...",0.0000000000000000000000000000000000000000000000000000000000000000000000000000000,❌ [False]
3,"{'A': 4.866467, 'B': 1.5433059, 'C': -1.1560992}",-0.158565,"Given the A,B,C coefficients, calculate the final value Y using the formula Y= -B/(2*A) of it. --- Follow the following format. Coefficients: list of float...",0.0000000000000000000000000000000000000000000000000000000000000000000000000000000,❌ [False]
4,"{'A': 2.607219, 'B': -4.0375066, 'C': 2.6858366}",0.774294,"Given the A,B,C coefficients, calculate the final value Y using the formula Y= -B/(2*A) of it. --- Follow the following format. Coefficients: list of float...",0.0000000000000000000000000000000000000000000000000000000000000000000000000000000,❌ [False]


6.25